## WRF Test Case Study: [Major Snowstorm December 16-17, 2020](https://www.weather.gov/aly/Snow17Dec20)

###  WPS: Model Domain Initialize
Detailed workflow [here](https://www2.mmm.ucar.edu/wrf/users/docs/user_guide_v4/v4.4/users_guide_chap3.html) \
Summary workflow [here](https://www2.mmm.ucar.edu/wrf/OnLineTutorial/compilation_tutorial.php)	

### Generate grid information

```{note}
Everytime `edit` is used in a command it implies you will use your chosen editing method. This could be via 'jupyterhub or terminal-based editor such as `emacs`
```

In WPS directory. Edit namelist.wps 
```
cd /glade/work/$USER/ASP2023/WPS
edit utils/namelist.wps 
```

Change the following lines only to the values shown (mostly getting rid of 2nd number and a `,` at end of each line)

```
max_dom = 1
parent_id = 1, 
parent_grid_ratio = 1, 
i_parent_start = 1, 
j_parent_start = 1, 
e_we = 74, 
e_sn = 61, 
geog_data_res = 'default',
```

Plot domain grid (requires a simple [ncl](https://www.ncl.ucar.edu) command and a module load if not currently installed))

```
module load ncl
ncl < util/plotgrids_new.ncl
```

View created image file `wps_show_dom.png` in jupyterhub 

Generate surface data for this domain (takes 1 minute)
```
./geogrid.exe	
```
Successful if last line on screen is
```
!  Successful completion of geogrid.        !
```








Data should be in `geo_em.d01.nc` (check it exists)
Browse data (should for the most part be obviously SE USA)  
```
ncview geo_em.d01.nc
```
Variable descriptions very limited. Try ncdump to look at the descriptions for each variable.
```
ncdump -h geo_em.d01.nc | less
```




Make a data directory (./DATA)
```
mkdir ~/WRF/DATA 
```
Grab domain datasets for this case study

Raw grib files used to initialize WRF forecasts are used [here](https://rda.ucar.edu/datasets/ds084.1/)


```
cp /glade/p/cgd/asp2017/WRF/JAN00_AWIP.tar.gz ~/WRF/DATA
```
Unzip/untar
```
cd ~/WRF/DATA
gzip -d JAN00_AWIP.tar.gz
tar -xvf JAN00_AWIP.tar -C ~/WRF/DATA 
```
Quick checking/listing (from ~/WRF/WPS directory) - Of initial variables
```
./util/g1print.exe ../DATA/JAN00/2000012412.AWIPSF
```
Data file linking 
```
ln -sf ungrib/Variable_Tables/Vtable.AWIP Vtable
./link_grib.csh ../DATA/JAN00/2000012
```
Edit namelist for dating info. 		emacs namelist.wps
```
run_hours = 36	
start_date = '2000-01-24_12:00:00',   
end_date = '2000-01-26_00:00:00',
```

### WRF: Running the model

Change to WRF directory from WPS directory
```
cd ../WRF
```
Let’s run the case here
```
cd test/em_real
```
And similar to linking the files we generated with WPS do the following
```
ln -sf ../../../WPS/met_em.d01.2000-01* .
```
Look over namelist
```	
emacs namelist.input 
```
We are only using the first column of values as we only have a single domain and no nesting
Extend end date of forecast out to **Jan 26 00Z**.
And change the gravity wave drag option to gwd_opt=0 instead	

In preparation for sensitivity runs check out descriptions of namelist variables
```
more README.namelist
```


### Sensitivity Experiments
